In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime
import nltk
from nltk.tokenize import sent_tokenize
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Data Processing
In order to do this market analysis, we need the following input data: 
1. Original transcripts from CoCoHD split into sentences
2. Energy-related keyword list
3. Relevancy Classifier result
4. IncDec Classifier result

In [ ]:
# Define path for input data files here
# Here we shows some example path we used while doing the market analysis
# please replace with the correct path you use to access these files

transcript_path = '/home/congress-user/data_collection/transcripts.csv'
keyword_list_path = '../data/target_word_list.csv'
relevancy_result_path = '../data/analysis_data/relevancy_full_labeled.csv'
incdec_result_path = '../data/analysis_data/IncDec_full_labeled.csv'

In [ ]:
# First, we process the raw transcripts into proper sentences with the nltk package
df_transcript = pd.read_csv(transcript_path)
df_transcript['sentences'] = df_transcript['content'].astype(str).apply(sent_tokenize)
df_transcript['sentence_count'] = df_transcript['sentences'].str.len()
df_transcript = df_transcript[['date','hearing_num','sentences']]

all_sentences = df_transcript.explode('sentences').reset_index(drop=True)
all_sentences['sentences'] = all_sentences['sentences'].apply(lambda text:re.sub(r'[^a-zA-Z0-9\s]', '', text))
all_sentences['sentences'] = all_sentences['sentences'].str.replace('\n','')

In [ ]:
# Using the path defined above, read the corresponding dataframes with preliminary processing
# Raw data
keyword_list = pd.read_csv(keyword_list_path, header=None).rename(columns={0:'keyword'})

# Classifier results
relevancy_result = pd.read_csv(relevancy_result_path)
incdec_result = pd.read_csv(incdec_result_path)

# Filter raw sentences dataframe with the keyword list with regex
filtered_df = all_sentences.copy()
filtered_df = filtered_df[filtered_df['sentences'].str.lower().str.contains(fr"\b(?:{'|'.join(keyword_list['keyword'].values)})\b")]
filtered_df['sentences'] = filtered_df['sentences'].replace(r'\s+', ' ', regex=True)

# Inclination Measure Construction
In this section, we use the previous data to construct hearing-level inclination measure. Here are definitions of some of our calculated measures:
- **keyword_pct**: keyword sentence count divided by total sentence count in percentage, represents how many sentences contain our predefined keyword per hearing.
- **relevancy_pct**: relevant sentence count divided by total sentence count in percentage, represents how many relevant sentences each hearing has.
- **neutral_pct**, **inc_pct**, **dec_pct**: percentage of neutral, inclined and declined sentences per hearing in percentage.
- **inc_dec_diff**: the main measurement we used for analysis, which represents the difference between number of inclined and declined sentences divided by the total number of relevant sentences.

In [ ]:
hearing_dates = all_sentences[['date','hearing_num']].drop_duplicates().sort_values(by=['date','hearing_num'])
hearing_dates['date'] = pd.to_datetime(hearing_dates['date'])

# merging results
# sentence_count: number of sentences per hearing
df1 = all_sentences.groupby(by=['hearing_num'])['sentences'].count().reset_index()
df1 = df1.rename(columns={'sentences':'sentence_count'})

# keyword_count: number of sentences that contain at least one energy keyword per hearing
df2 = filtered_df.groupby(by=['hearing_num'])['sentences'].count().reset_index()
df2 = df2.rename(columns={'sentences':'keyword_count'})

# irrel_count: number of irrelevant sentences per hearing
df3 = relevancy_result[relevancy_result['label'] == 'i'].groupby(by=['hearing_num'])['sentence'].count().reset_index()
df3 = df3.rename(columns={'sentence':'irrel_count'})

# rel_count: number of relevant sentences per hearing
df4 = relevancy_result[relevancy_result['label'] == 'r'].groupby(by=['hearing_num'])['sentence'].count().reset_index()
df4 = df4.rename(columns={'sentence':'rel_count'})

# neutral_count: number of neutral sentences per hearing
df5 = incdec_result[incdec_result['label'] == 'n'].groupby(by=['hearing_num'])['sentence'].count().reset_index()
df5 = df5.rename(columns={'sentence':'neutral_count'})

# inc_count: number of inclined sentences per hearing
df6 = incdec_result[incdec_result['label'] == 'p'].groupby(by=['hearing_num'])['sentence'].count().reset_index()
df6 = df6.rename(columns={'sentence':'inc_count'})

# dec_count: number of declined sentences per hearing
df7 = incdec_result[incdec_result['label'] == 'd'].groupby(by=['hearing_num'])['sentence'].count().reset_index()
df7 = df7.rename(columns={'sentence':'dec_count'})

# result frame basics
result_df = df1.merge(df2, on='hearing_num', how='left').merge(df3, on='hearing_num', how='left').merge(df4, on='hearing_num', how='left').merge(df5, on='hearing_num', how='left').merge(df6, on='hearing_num', how='left').merge(df7, on='hearing_num', how='left')
result_df = result_df.fillna(0)
result_df['sentence_count'] = result_df['sentence_count'].astype(int)
result_df['keyword_count'] = result_df['keyword_count'].astype(int)
result_df['irrel_count'] = result_df['irrel_count'].astype(int)
result_df['rel_count'] = result_df['rel_count'].astype(int)
result_df['neutral_count'] = result_df['neutral_count'].astype(int)
result_df['inc_count'] = result_df['inc_count'].astype(int)
result_df['dec_count'] = result_df['dec_count'].astype(int)
result_df = result_df.merge(hearing_dates,on='hearing_num',how='left')
result_df['date'] = pd.to_datetime(result_df['date'])
result_df['year'] = result_df['date'].dt.to_period('Y')
result_df['month'] = result_df['date'].dt.to_period('M')
result_df['quarter'] = result_df['date'].dt.to_period('Q')

# indicator calculators
result_df['keyword_pct'] = result_df['keyword_count']*100/result_df['sentence_count']
result_df['relevancy_pct'] = result_df['rel_count']*100/result_df['sentence_count']
result_df['neutral_pct'] = result_df['neutral_count']*100/result_df['sentence_count']
result_df['inc_pct'] = result_df['inc_count']*100/result_df['sentence_count']
result_df['dec_pct'] = result_df['dec_count']*100/result_df['sentence_count']
result_df['inc_dec_diff'] = (result_df['inc_count']-result_df['dec_count'])*100/result_df['rel_count']
result_df = result_df.fillna(0)

# save measure as a dataframe
# result_df[['date','quarter','month','hearing_num','sentence_count','keyword_count','irrel_count',
#            'rel_count','neutral_count','inc_count','dec_count',
#            'keyword_pct','relevancy_pct','neutral_pct','inc_pct','dec_pct','inc_dec_diff']].to_csv('measures.csv')
# or display results
# result_df

# Regression Analysis
In this part, we conduct a quantitative analysis using the VDE Index to demonstrate the economic significance of our inclination measure in the energy market. As detailed in the paper, in order to filter our the strongest signals, we conduct some filtering first to identify hearings held during period of high momentum while having prominent relevancy to energy-related discourses. Thus, we use the following rules:
1. Count of relevant sentences >= 22 (70% percentile of the relevancy distribution)
2. Relative strength index (RSI) >= 70 or <= 30 (this measures the speed and change of price movements in financial markets, indicating overbought and oversold signals)
3. Volatility (previous 7-days) >= 0.012 (50% precentile of the volatility distribution)

## VDE Benchmark
First, we read in the VDE historic data and calculate some indicators such as volatility, RSI, and returns over different windows to support further analysis.

In [ ]:
# Here's an example path we use for the VDE data
# Replace with your customised directory
vde_df_path = '/home/congress-user/data_collection/energy_data/vde.csv'

In [ ]:
vde_df = pd.read_csv(vde_df_path, index_col=0)
vde_df['Date'] = pd.to_datetime(vde_df['Date'])
vde_df = vde_df.rename(columns={'Close':'PRICE','Date':'DATE'})
vde_df = vde_df.drop(columns={'Open','High','Low','Adj Close'})
vde_df['PRICE'] = pd.to_numeric(vde_df['PRICE'], errors='coerce')
vde_df['year'] = vde_df['DATE'].dt.year
vde_df = vde_df[['DATE','year','PRICE','Volume']]

def rsi_calc(df):
    change = df['PRICE'].diff()
    change.dropna(inplace=True)
    change_up = change.copy()
    change_down = change.copy()
    change_up[change_up<0] = 0
    change_down[change_down>0] = 0
    change.equals(change_up+change_down)
    
    avg_up = change_up.rolling(14).mean()
    avg_down = change_down.rolling(14).mean().abs()
    rsi = 100*avg_up/(avg_up+avg_down)
    return rsi

# calculate necessary indicators
vde_df['returns'] = vde_df['PRICE'].pct_change()
vde_df['log_ret'] = np.log(vde_df['PRICE']) - np.log(vde_df['PRICE'].shift(1))
vde_df['volatility'] = vde_df['log_ret'].rolling(window=7).std()
vde_df['prev7_volatility'] = vde_df['volatility'].shift(7)
vde_df['rsi'] = rsi_calc(vde_df)

vde_df['RETURN_1'] = vde_df['PRICE'].pct_change(1).shift(-1)
vde_df['RETURN_2'] = vde_df['PRICE'].pct_change(2).shift(-2)
vde_df['RETURN_3'] = vde_df['PRICE'].pct_change(3).shift(-3)
vde_df['RETURN_4'] = vde_df['PRICE'].pct_change(4).shift(-4)
vde_df['RETURN_5'] = vde_df['PRICE'].pct_change(5).shift(-5)
vde_df['RETURN_6'] = vde_df['PRICE'].pct_change(6).shift(-6)
vde_df['RETURN_7'] = vde_df['PRICE'].pct_change(7).shift(-7)
vde_df['RETURN_14'] = vde_df['PRICE'].pct_change(14).shift(-14)

vde_df['VOLATILITY_7'] = vde_df['volatility'].shift(-7)
vde_df['VOLATILITY_14'] = vde_df['volatility'].shift(-14)
vde_df['VOLATILITY_21'] = vde_df['volatility'].shift(-21)
vde_df['VOLATILITY_28'] = vde_df['volatility'].shift(-28)

vde_df = vde_df[~vde_df['prev7_volatility'].isna()]
vde_df = vde_df[~vde_df['rsi'].isna()]

## Filtering

In [ ]:
# criteria1: relevant sentence count, threshold = 22 (70%)
analysis = result_df.copy()
analysis = analysis[analysis['rel_count'] >= 22]
merged_df = analysis.merge(vde_df, left_on='date',right_on='DATE')

# criteria2: RSI >= 70 or RSI <= 30
filtered = merged_df[(merged_df['rsi'] >= 70) | (merged_df['rsi'] <= 30)].copy()

# criteria3: prev7_volatility >= 0.012
filtered = filtered[filtered['prev7_volatility'] >= 0.012].copy()

## Visualization

In [ ]:
# visualizing relation between our measurement and returns
# threshold: vol - 0.12, rsi - 30/70, rel - 22
fig, axes = plt.subplots(nrows=1, ncols=5,figsize=(30,4))

sns.regplot(x='inc_dec_diff', y='RETURN_1', data=filtered, robust=True, ax=axes[0])
sns.regplot(x='inc_dec_diff', y='RETURN_5', data=filtered, robust=True, ax=axes[1])
sns.regplot(x='inc_dec_diff', y='RETURN_6', data=filtered, robust=True, ax=axes[2])
sns.regplot(x='inc_dec_diff', y='RETURN_7', data=filtered, robust=True, ax=axes[3])
sns.regplot(x='inc_dec_diff', y='RETURN_14', data=filtered, robust=True, ax=axes[4])

In [ ]:
# visualizing relation between our measurement and volatilities
# threshold: vol - 0.12, rsi - 30/70, rel - 22
fig, axes = plt.subplots(nrows=2, ncols=3,figsize=(16,7))

sns.regplot(x='inc_dec_diff', y='VOLATILITY_5', data=filtered, robust=True, ax=axes[0][0])
sns.regplot(x='inc_dec_diff', y='VOLATILITY_7', data=filtered, robust=True, ax=axes[0][1])
sns.regplot(x='inc_dec_diff', y='VOLATILITY_14', data=filtered, robust=True, ax=axes[0][2])
sns.regplot(x='inc_dec_diff', y='VOLATILITY_21', data=filtered, robust=True, ax=axes[1][0])
sns.regplot(x='inc_dec_diff', y='VOLATILITY_28', data=filtered, robust=True, ax=axes[1][1])
axes[1,2].set_visible(False)

## Regression
In this section, we conduct regression analysis between returns, volatilities and the inclination measure, and report correpsonding statistics. 

In [34]:
smf.ols(formula='RETURN_5 ~ inc_dec_diff', data=filtered).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               RETURN_5   R-squared:                       0.094
Model:                            OLS   Adj. R-squared:                  0.067
Method:                 Least Squares   F-statistic:                     3.533
Date:                Sun, 02 Jun 2024   Prob (F-statistic):             0.0688
Time:                        19:53:02   Log-Likelihood:                 67.000
No. Observations:                  36   AIC:                            -130.0
Df Residuals:                      34   BIC:                            -126.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0040      0.008      0.522      0.605      -0.012       0.019
inclination_measure     0.0005      0.000      1.880      0.069   -3.84e-05       0.001
==============================================================================
Omnibus:                        5.708   Durbin-Watson:                   1.627
Prob(Omnibus):                  0.058   Jarque-Bera (JB):                4.302
Skew:                           0.643   Prob(JB):                        0.116
Kurtosis:                       4.102   Cond. No.                         35.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
smf.ols(formula='RETURN_6 ~ inc_dec_diff', data=filtered).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               RETURN_6   R-squared:                       0.147
Model:                            OLS   Adj. R-squared:                  0.122
Method:                 Least Squares   F-statistic:                     5.845
Date:                Sun, 02 Jun 2024   Prob (F-statistic):             0.0211
Time:                        19:53:04   Log-Likelihood:                 60.871
No. Observations:                  36   AIC:                            -117.7
Df Residuals:                      34   BIC:                            -114.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0039      0.009      0.435      0.666      -0.014       0.022
inclination_measure     0.0007      0.000      2.418      0.021       0.000       0.001
==============================================================================
Omnibus:                        0.045   Durbin-Watson:                   1.940
Prob(Omnibus):                  0.978   Jarque-Bera (JB):                0.064
Skew:                          -0.040   Prob(JB):                        0.968
Kurtosis:                       2.809   Cond. No.                         35.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [36]:
smf.ols(formula='RETURN_7 ~ inc_dec_diff', data=filtered).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               RETURN_7   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     2.804
Date:                Sun, 02 Jun 2024   Prob (F-statistic):              0.103
Time:                        19:53:06   Log-Likelihood:                 57.858
No. Observations:                  36   AIC:                            -111.7
Df Residuals:                      34   BIC:                            -108.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0059      0.010      0.603      0.550      -0.014       0.026
inclination_measure     0.0005      0.000      1.674      0.103      -0.000       0.001
==============================================================================
Omnibus:                        0.009   Durbin-Watson:                   1.742
Prob(Omnibus):                  0.995   Jarque-Bera (JB):                0.126
Skew:                           0.033   Prob(JB):                        0.939
Kurtosis:                       2.718   Cond. No.                         35.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [84]:
smf.ols(formula='VOLATILITY_28 ~ inc_dec_diff', data=filtered).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VOLATILITY_28   R-squared:                       0.209
Model:                            OLS   Adj. R-squared:                  0.186
Method:                 Least Squares   F-statistic:                     8.990
Date:                Wed, 03 Apr 2024   Prob (F-statistic):            0.00505
Time:                        18:15:15   Log-Likelihood:                 31.935
No. Observations:                  36   AIC:                            -59.87
Df Residuals:                      34   BIC:                            -56.70
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.2791      0.020     13.837      0.000       0.238       0.320
sentiment_measure    -0.0020      0.001     -2.998      0.005      -0.003      -0.001
==============================================================================
Omnibus:                        1.568   Durbin-Watson:                   1.898
Prob(Omnibus):                  0.457   Jarque-Bera (JB):                1.497
Skew:                           0.433   Prob(JB):                        0.473
Kurtosis:                       2.503   Cond. No.                         35.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [87]:
smf.ols(formula='VOLATILITY_21 ~ inc_dec_diff', data=filtered).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VOLATILITY_21   R-squared:                       0.157
Model:                            OLS   Adj. R-squared:                  0.132
Method:                 Least Squares   F-statistic:                     6.333
Date:                Wed, 03 Apr 2024   Prob (F-statistic):             0.0167
Time:                        18:15:47   Log-Likelihood:                 32.893
No. Observations:                  36   AIC:                            -61.79
Df Residuals:                      34   BIC:                            -58.62
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.2812      0.020     14.320      0.000       0.241       0.321
sentiment_measure    -0.0016      0.001     -2.517      0.017      -0.003      -0.000
==============================================================================
Omnibus:                        3.525   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.172   Jarque-Bera (JB):                2.997
Skew:                           0.703   Prob(JB):                        0.223
Kurtosis:                       2.861   Cond. No.                         35.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [85]:
smf.ols(formula='VOLATILITY_14 ~ inc_dec_diff', data=filtered).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VOLATILITY_14   R-squared:                       0.139
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     5.481
Date:                Wed, 03 Apr 2024   Prob (F-statistic):             0.0252
Time:                        18:15:20   Log-Likelihood:                 32.852
No. Observations:                  36   AIC:                            -61.70
Df Residuals:                      34   BIC:                            -58.54
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.2818      0.020     14.335      0.000       0.242       0.322
sentiment_measure    -0.0015      0.001     -2.341      0.025      -0.003      -0.000
==============================================================================
Omnibus:                        4.697   Durbin-Watson:                   2.045
Prob(Omnibus):                  0.096   Jarque-Bera (JB):                3.948
Skew:                           0.811   Prob(JB):                        0.139
Kurtosis:                       3.024   Cond. No.                         35.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [86]:
smf.ols(formula='VOLATILITY_7 ~ inc_dec_diff', data=filtered).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           VOLATILITY_7   R-squared:                       0.111
Model:                            OLS   Adj. R-squared:                  0.085
Method:                 Least Squares   F-statistic:                     4.249
Date:                Wed, 03 Apr 2024   Prob (F-statistic):             0.0470
Time:                        18:15:29   Log-Likelihood:                 20.202
No. Observations:                  36   AIC:                            -36.40
Df Residuals:                      34   BIC:                            -33.24
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.2744      0.028      9.819      0.000       0.218       0.331
sentiment_measure    -0.0019      0.001     -2.061      0.047      -0.004   -2.68e-05
==============================================================================
Omnibus:                        3.258   Durbin-Watson:                   2.047
Prob(Omnibus):                  0.196   Jarque-Bera (JB):                2.958
Skew:                           0.679   Prob(JB):                        0.228
Kurtosis:                       2.645   Cond. No.                         35.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Market Analysis
In this section, we perform a qualitative analysis of our inclination measure, by comparing its overall trend with Google Trends for "clearn enregy" and "fossil fuel" keywords.

## Clean Energy

In [ ]:
# define the path to clean energy search data, downloaded from Google Trends
clean_energy_search_path = '/home/congress-user/data_collection/search_data/clean_energy_search.csv'

clean_energy_search = pd.read_csv(clean_energy_search_path)
clean_energy_search = clean_energy_search.iloc[1:]
clean_energy_search = clean_energy_search.rename(columns={'Category: All categories':'#search'})
clean_energy_search['month'] = pd.to_datetime(clean_energy_search.index).to_period('M')
clean_energy_search['#search'] = clean_energy_search['#search'].astype(int)

grouped_clean = result_df.groupby(['month'])['inc_dec_diff'].sum().reset_index()
grouped_clean = grouped_clean.merge(clean_energy_search)

marked_dates_dec = grouped_clean[grouped_clean['inc_dec_diff'] < 0]['month'].unique()
marked_dates_inc = grouped_clean[grouped_clean['inc_dec_diff'] > 0]['month'].unique()
search_marked_inc = clean_energy_search[clean_energy_search['month'].isin(marked_dates_inc)]
search_marked_dec = clean_energy_search[clean_energy_search['month'].isin(marked_dates_dec)]

ax = clean_energy_search.plot(x='month',y='#search',figsize=(15,4), alpha=0.7)
search_marked_inc.reset_index().plot.scatter(x = 'index', y = '#search',c='r',ax=ax)
search_marked_dec.reset_index().plot.scatter(x = 'index', y = '#search',c='g',ax=ax)
plt.title('clean_energy search frequency marked by the measurement - aggregated by month')
plt.show()

## fossil fuel

In [ ]:
# define the path to clean energy search data, downloaded from Google Trends
fossil_fuel_search_path = '/home/congress-user/data_collection/search_data/fossil_fuel_search.csv'

fossil_fuel_search = pd.read_csv(fossil_fuel_search_path)
fossil_fuel_search = fossil_fuel_search.iloc[1:]
fossil_fuel_search = fossil_fuel_search.rename(columns={'Category: All categories':'#search'})
fossil_fuel_search['month'] = pd.to_datetime(fossil_fuel_search.index).to_period('M')
fossil_fuel_search['#search'] = fossil_fuel_search['#search'].astype(int)

grouped_fossil = result_df.groupby(['month'])['inc_dec_diff'].sum().reset_index()
grouped_fossil = grouped_fossil.merge(fossil_fuel_search)

marked_dates_dec = grouped_fossil[grouped_fossil['inc_dec_diff'] < 0]['month'].unique()
marked_dates_inc = grouped_fossil[grouped_fossil['inc_dec_diff'] > 0]['month'].unique()
search_marked_inc = fossil_fuel_search[fossil_fuel_search['month'].isin(marked_dates_inc)]
search_marked_dec = fossil_fuel_search[fossil_fuel_search['month'].isin(marked_dates_dec)]

ax = fossil_fuel_search.plot(x='month',y='#search',figsize=(15,4), alpha=0.7)
search_marked_inc.reset_index().plot.scatter(x = 'index', y = '#search',c='r',ax=ax)
search_marked_dec.reset_index().plot.scatter(x = 'index', y = '#search',c='g',ax=ax)
plt.title('fossil_fuel search frequency marked by the measurement - aggregated by month')
plt.show()